In [1]:
import numpy as np
import random
from scipy.linalg import fractional_matrix_power


def generate_random_Y(k, N):
    Y = np.zeros((N, k))
    for i in range(N):
        j = random.randint(0, k-1)
        Y[i][j] = 1
    return Y

def target(Y, D, L):
    tmp = np.dot(Y.T, np.dot(D, Y))
    try:
        tmp_inv = np.linalg.inv(tmp)
    except np.linalg.LinAlgError:
        return 1e9  # 当矩阵不可逆时返回 1e9
    return np.trace(np.dot(tmp_inv, np.dot(Y.T, np.dot(L, Y))))

def minimize(Y, A):
    D = np.diag(np.sum(A, axis=1))
    L = D - A
    return target(Y, D, L)

def D_uv_from_D(D, m, n):
    '''
    Du, Dv = D_uv_from_D(D, m, n)
    '''
    Du = np.diag(np.sum(D, axis=1)[:m])
    Dv = np.diag(np.sum(D, axis=1)[m:])
    return Du, Dv

In [2]:
B = np.array(
    [
        [0, 0, 1, 1, 0],
        [0, 1, 1, 0, 0],
        [0, 0, 2, 2, 0],
        [0, 1, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 0, 3, 3, 0],
    ]
)

# A = 0 B
#     B^T 0

# 获取 A 的转置
B_T = B.T

m, n = B.shape
upper_zeros = np.zeros((m, m))
right_zeros = np.zeros((n, n))

A = np.vstack((np.hstack((upper_zeros, B)), np.hstack((B_T, right_zeros)))) # (n+m) * (n+m), N := n+m
# print(A)

# A= [[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
#     [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
#     [0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0.]
#     [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
#     [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
#     [0. 0. 0. 0. 0. 0. 0. 0. 3. 3. 0.]
#     [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
#     [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
#     [1. 1. 2. 1. 0. 3. 0. 0. 0. 0. 0.]
#     [1. 0. 2. 0. 0. 3. 0. 0. 0. 0. 0.]
#     [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]

D = np.diag(np.sum(A, axis=1))

L = D - A

# L
k = 3 ## number of clusters
cut = [] ## 1 * k, cut[i] = all weights across cluster i
asso = [] ## 1 * k, asso[i] = all weights that one side is in cluster i

Y = [] ## N * k, Y[i][j] = 1 if node i is in j

# random a Y
N = n + m


Y = generate_random_Y(k, N)

tmp = np.dot(Y.T, np.dot(D, Y))

Z = np.dot(Y, fractional_matrix_power(tmp, -0.5))

lfs = np.trace(np.dot(Z.T, np.dot(L, Z)))

# (Y^T D Y)^(-1)Y^T L Y

# rfs = np.trace(np.dot(np.linalg.inv(tmp), np.dot(Y.T, np.dot(L, Y))))


rfs = minimize(Y, A)

ld = np.trace(np.dot(L, np.linalg.inv(D)))

lfs, rfs, ld, tmp


(1.9999999999999996,
 1.9999999999999998,
 11.0,
 array([[18.,  0.,  0.],
        [ 0.,  9.,  0.],
        [ 0.,  0.,  9.]]))

In [2]:


# 穷举所有的 Y
min_value = 1e9

for i in range(k ** N):
    Y = np.zeros((N, k))
    for j in range(N):
        Y[j][i // (k ** j) % k] = 1
    
    value = minimize(Y, A)
    if value < min_value:
        min_value = value
        min_Y = Y
        
min_value, min_Y

(0.41025641025641024,
 array([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.]]))

In [5]:
min_Dp = np.dot(min_Y.T, np.dot(D, min_Y))

min_Dp

array([[ 4.,  0.,  0.],
       [ 0., 26.,  0.],
       [ 0.,  0.,  6.]])

In [3]:
min_Z = np.dot(min_Y, fractional_matrix_power(np.dot(min_Y.T, np.dot(D, min_Y)), -0.5))
F = np.dot(fractional_matrix_power(D, 0.5), min_Z)

F

array([[0.        , 0.2773501 , 0.        ],
       [0.        , 0.        , 0.57735027],
       [0.        , 0.39223227, 0.        ],
       [0.        , 0.        , 0.57735027],
       [0.70710678, 0.        , 0.        ],
       [0.        , 0.48038446, 0.        ],
       [0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.57735027],
       [0.        , 0.5547002 , 0.        ],
       [0.        , 0.48038446, 0.        ],
       [0.5       , 0.        , 0.        ]])

In [28]:
# check when Z^T D Z is identity matrix
Y = generate_random_Y(k, N)

# Y 的第二行*2
Y[1] = Y[1] * 2

# Y 的第一列*8
Y[:, 0] = Y[:, 0] * 8

# Y 的最后一行是 0 0 0
Y[-1] = 0

def Z_from_Y_D(Y, D):
    Dp = np.dot(Y.T, np.dot(D, Y))
    return np.dot(Y, fractional_matrix_power(Dp, -0.5))

Z = Z_from_Y_D(Y, D)
Y, np.dot(Z.T, np.dot(D, Z)), np.dot(Y.T, Y)

(array([[0., 0., 1.],
        [0., 0., 2.],
        [0., 1., 0.],
        [8., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [8., 0., 0.],
        [0., 0., 0.]]),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[128.,   0.,   0.],
        [  0.,   3.,   0.],
        [  0.,   0.,   8.]]))

In [4]:
Du, Dv = D_uv_from_D(D, m, n)

svd_source = np.dot(fractional_matrix_power(Du, -0.5), np.dot(B, fractional_matrix_power(Dv, -0.5)))

u, s, vh = np.linalg.svd(svd_source)

U = np.sqrt(2)/2 * u[:, :k]
Vh = np.sqrt(2)/2 * vh[:k, :]

# U, Vh.T, np.dot(U.T, U) + np.dot(Vh, Vh.T)

# F = [U;V]
F = np.vstack((U, Vh.T))

# F
svd_source

array([[0.        , 0.        , 0.25      , 0.28867513, 0.        ],
       [0.        , 0.5       , 0.25      , 0.        , 0.        ],
       [0.        , 0.        , 0.35355339, 0.40824829, 0.        ],
       [0.        , 0.5       , 0.25      , 0.        , 0.        ],
       [0.70710678, 0.        , 0.        , 0.        , 0.70710678],
       [0.        , 0.        , 0.4330127 , 0.5       , 0.        ]])

In [49]:
np.dot(min_Y, min_Y.T), min_Y

(array([[1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.],
        [0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.],
        [0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.]]),
 array([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.]]))

In [57]:
# 对 F 的每一行作为一个样本，进行k-means聚类

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=k, random_state=0).fit(F)

label = kmeans.labels_

labele_u = label[:m]
labele_v = label[m:]

labele_u, labele_v

(array([1, 2, 1, 2, 0, 1], dtype=int32), array([0, 2, 1, 1, 0], dtype=int32))

In [59]:
# 按照聚类结果，将 B 重新排列

vis_B = B[labele_u.argsort()]
vis_B = vis_B[:, labele_v.argsort()]

vis_B

array([[1, 1, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 3, 3, 0],
       [0, 0, 2, 2, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 0, 1, 1]])